In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from pathlib import Path
import pandas as pd
import numpy as np
from spotifuncs import *

In [2]:
path = Path("C:/Users/ms101/OneDrive/DataScience_ML/projects/spotify_app")

In [3]:
with open(path / "client_s.txt") as f:
    content = f.readlines()
content = [x.strip() for x in content]

client_id = content[0]
client_secret = content[1]

In [4]:
with open(path / "usernames.txt") as f:
    usernames = f.readlines()
usernames = [x.strip() for x in usernames]

username1 = usernames[0]

In [5]:
scope = "user-library-read user-read-recently-played user-top-read playlist-modify-public playlist-modify-private playlist-read-private playlist-read-collaborative"

redirect_uri = "https://developer.spotify.com/dashboard/applications/4a4e029d299a4241873db8300038bf0a"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
                                                      client_secret=client_secret)


In [6]:
sp_m = authenticate(redirect_uri, client_credentials_manager, username1, scope, client_id, client_secret)

In [7]:
redlight = sp_m.playlist("spotify:playlist:3zcSUFp0puoWXWXuFCF2e6")

In [8]:
# def create_df_playlist(api_results,sp = None, append_audio = True):
#     df = create_df_saved_songs(api_results["tracks"])
#     if append_audio == True:
#         assert sp != None, "sp needs to be specified for appending audio features"
#         df = append_audio_features(df,sp)
#     return df

In [9]:
redlight_df = create_df_playlist(redlight)#see if assert error works

AssertionError: sp needs to be specified for appending audio features

In [10]:
redlight_df = create_df_playlist(redlight, sp = sp_m)
redlight_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Desert Woman,RINGOS DESERT,4O1nIKXTS87DKHIQejrn3n,ZHU,374610,44,0.710,0.651,1,-8.537,1,0.0707,0.022300,0.864,0.0657,0.1500,122.987
1,York - Original Mix,York,3eJBKLhpOauQJlgoOSPErD,Christian Löffler,476675,49,0.696,0.634,11,-12.063,0,0.0403,0.157000,0.887,0.1290,0.0359,118.035
2,Personal Space,Personal Space / Mulholland 99,3TYNQdnAM85I6P9UegzYKo,Yotto,301295,46,0.645,0.848,7,-6.913,1,0.0351,0.002540,0.956,0.0868,0.0383,122.011
3,October,October,6FIQ8o2hqlDmHQFoBKmKgW,Icarus,354102,59,0.365,0.711,8,-9.591,1,0.0355,0.030600,0.913,0.0824,0.1170,116.003
4,Azure,Berlin Calling (The Soundtrack by Paul Kalkbre...,2VnlJCQMphFJUyYR5p7da2,Paul Kalkbrenner,364059,57,0.730,0.504,4,-13.915,1,0.0477,0.124000,0.882,0.1070,0.0936,122.013
5,Eating Hooks - Siriusmo Remix - Solomun Edit,Eating Hooks (Siriusmo Remix - Solomun Edit),6fgmNIMwDdLQrNzu4U8pcU,Moderat,422200,55,0.661,0.728,8,-8.713,1,0.0379,0.008180,0.652,0.0599,0.3470,123.994
6,Parallaxis - Traumprinz's Over 2 The End Version,Decay Versions Pt. 2,6o6LRX4HHHGKgH7NuMxw6u,Efdemin,406000,42,0.688,0.496,7,-15.994,1,0.0519,0.076400,0.894,0.1170,0.2030,122.971
7,Mango,Mango,1lK8OlJuuoVsGHsWaKFAfk,Sascha Funke,420919,46,0.670,0.548,5,-11.342,0,0.0571,0.000724,0.918,0.0857,0.4380,124.996
8,Risky Business,Risky Business,75N5R8CeZDyJurQJsoRTx1,ZHU,320853,58,0.787,0.653,11,-6.825,0,0.0649,0.087500,0.735,0.0933,0.3960,122.995
9,Little by Little,Little by Little,2XgcOGlE73KWYa43Deq8OJ,Lane 8,333223,54,0.649,0.771,9,-9.714,1,0.0412,0.008590,0.853,0.1030,0.6080,121.011


In [11]:
cols = redlight_df.columns[6:].tolist()
cols

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo']

In [12]:
redlight_df[cols].mean()

danceability          0.722836
energy                0.556091
key                   6.090909
loudness            -11.632145
mode                  0.509091
speechiness           0.057227
acousticness          0.148777
instrumentalness      0.853545
liveness              0.108858
valence               0.235367
tempo               119.640327
dtype: float64

In [13]:
mean_song = pd.DataFrame(columns=redlight_df.columns)
mean_song.loc["mean"] = redlight_df.mean()

In [14]:
mean_song

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
mean,NaN,NaN,NaN,NaN,407057.436364,37.254545,0.722836,0.556091,6.090909,-11.632145,0.509091,0.057227,0.148777,0.853545,0.108858,0.235367,119.640327


In [15]:
seed_tracks = redlight_df["track_id"].tolist()

In [16]:
len(seed_tracks)

55

In [17]:
recomm_dfs = []
for i in range(5,len(seed_tracks)+1,5):
    recomms = sp_m.recommendations(seed_tracks = seed_tracks[i-5:i],limit = 25)
    recomms_df = append_audio_features(create_df_recommendations(recomms),sp_m)
    recomm_dfs.append(recomms_df)
recomms_df = pd.concat(recomm_dfs)
recomms_df.reset_index(drop = True, inplace = True)

In [18]:
recomms_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Neck And Neck - DNMO Remix,Northern Lights (Remixes),2bCjaF7cRm53Wdq9RtkYJu,Zeds Dead,182400,63,0.601,0.591,5,-7.407,0,0.0337,0.018800,0.076600,0.1260,0.3700,150.044
1,Spiritual but Not Religious,The Arc of Tension,6fxq63aKjMmi1WMI9iKX5w,Oliver Koletzki,246706,58,0.671,0.497,0,-9.787,0,0.0552,0.171000,0.536000,0.1010,0.1140,95.005
2,Nightcrawler,Nightcrawler,4YixJQ4RxZnTBPNCeICR25,ZHU,270930,45,0.866,0.414,2,-9.405,1,0.0516,0.109000,0.876000,0.1040,0.2350,122.015
3,go with u,Someone's EP II,4PdLup4OaCGktZG8cvbkBz,BAYNK,231192,67,0.787,0.537,1,-6.827,0,0.2040,0.594000,0.000063,0.1140,0.3460,108.994
4,Flames (feat. Ruel) - Lastlings Remix,Flames (feat. Ruel) [Lastlings Remix],5GSxCRwcmFUo0eYjV5UmlR,SG Lewis,258647,52,0.737,0.613,11,-8.482,0,0.0475,0.012600,0.692000,0.1430,0.0943,123.990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,Safe,Safe,7v2iUsHqHR1VNLODNUkTOz,Monkey Safari,501649,55,0.648,0.725,1,-8.806,1,0.0325,0.000169,0.834000,0.1070,0.6050,123.999
271,Solitary Native,Solitary Native / Old Flame,4nliTt7jzYzGhSdrC8ZGnW,Alix Perez,451226,43,0.595,0.758,10,-15.259,0,0.0656,0.015500,0.877000,0.0699,0.4850,172.027
272,Do Not Do Me (Like Dis),Do Not Do Me (Like Dis),4DGrOp8YhMx5WocfOd0pQE,Moonbootica,194081,27,0.774,0.818,8,-6.445,1,0.0502,0.018200,0.012600,0.0884,0.4340,119.009
273,Halo 22,Eclipse,3VMwehRQtTt7gXAFirRW6t,Emmit Fenn,214372,35,0.731,0.464,0,-15.095,0,0.0580,0.844000,0.963000,0.0782,0.0703,120.034


In [19]:
#testing whether MinMax Scaling works better than StandardScaling
from sklearn.preprocessing import MinMaxScaler
def test_create_similarity_score(df1,df2,similarity_score = "cosine_sim"):
    """ 
    Creates a similarity matrix for the audio features (except key and mode) of two Dataframes.

    Parameters
    ----------
    df1 : DataFrame containing track_name,track_id, artist,album,duration,popularity
            and all audio features
    df2 : DataFrame containing track_name,track_id, artist,album,duration,popularity
            and all audio features
    
    similarity_score: similarity measure (linear,cosine_sim)

    Returns
    -------
    A matrix of similarity scores for the audio features of both DataFrames.
    """
    
    assert list(df1.columns[6:]) == list(df2.columns[6:]), "dataframes need to contain the same columns"
    features = list(df1.columns[6:])
    features.remove('key')
    features.remove('mode')
    df_features1,df_features2 = df1[features],df2[features]
    scaler = MinMaxScaler()
    df_features_scaled1,df_features_scaled2 = scaler.fit_transform(df_features1),scaler.fit_transform(df_features2)
    if similarity_score == "linear":
        linear_sim = linear_kernel(df_features_scaled1, df_features_scaled2)
        return linear_sim
    elif similarity_score == "cosine_sim":
        cosine_sim = cosine_similarity(df_features_scaled1, df_features_scaled2)
        return cosine_sim
    #other measures may be implemented in the future


In [20]:
similarity_score = create_similarity_score(redlight_df,recomms_df)
test_similarity_score = test_create_similarity_score(redlight_df,recomms_df)

In [21]:
similarity_score.shape, test_similarity_score.shape

((55, 275), (55, 275))

In [22]:
[np.argmax(i) for i in similarity_score], [np.argmax(i) for i in test_similarity_score]

([233,
  174,
  105,
  261,
  148,
  43,
  65,
  61,
  35,
  116,
  272,
  58,
  114,
  62,
  94,
  1,
  130,
  139,
  208,
  11,
  45,
  172,
  233,
  156,
  114,
  169,
  168,
  233,
  82,
  147,
  159,
  94,
  254,
  169,
  119,
  208,
  168,
  67,
  247,
  76,
  87,
  117,
  225,
  83,
  267,
  213,
  65,
  87,
  132,
  62,
  235,
  103,
  124,
  13,
  193],
 [233,
  29,
  37,
  217,
  81,
  152,
  81,
  61,
  228,
  80,
  34,
  81,
  114,
  62,
  183,
  1,
  112,
  146,
  150,
  11,
  34,
  155,
  1,
  155,
  114,
  155,
  169,
  114,
  82,
  2,
  89,
  183,
  112,
  155,
  119,
  66,
  168,
  67,
  81,
  167,
  50,
  1,
  167,
  11,
  267,
  213,
  250,
  78,
  132,
  66,
  191,
  24,
  169,
  180,
  193])

In [23]:
final_recomms = recomms_df.iloc[[np.argmax(i) for i in similarity_score]]
final_recomms = final_recomms.drop_duplicates().reset_index(drop = True)

In [24]:
test_final_recomms = recomms_df.iloc[[np.argmax(i) for i in test_similarity_score]]
test_final_recomms = test_final_recomms.drop_duplicates().reset_index(drop = True)

In [25]:
final_recomms

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,For an Endless Night - Jel Ford Remix,For an Endless Night (Jel Ford Remix),39KN3wgqTrGFxvWRyMW5zL,Alan Fitzpatrick,449768,38,0.751,0.649,8,-6.885,0,0.0579,0.015100,0.761000,0.0976,0.1790,125.996
1,York - Original Mix,York,3eJBKLhpOauQJlgoOSPErD,Christian Löffler,476675,49,0.696,0.634,11,-12.063,0,0.0403,0.157000,0.887000,0.1290,0.0359,118.035
2,AC1609 - Edit,AC1609 (Edit),0fKfg36ICmmfYBGAtqJ0Yh,Kölsch,272487,29,0.611,0.820,2,-9.247,1,0.0420,0.104000,0.927000,0.0949,0.0630,124.989
3,Up All Night - Radio Edit,Up All Night,60OOnXwxebQ4nysceVgQZX,Agoria,182787,30,0.562,0.719,11,-8.338,0,0.0328,0.060300,0.745000,0.0944,0.2260,121.009
4,Vierfleck - Recondite Remix,Mannigfaltig Remixes (Pt. 2),3i7PKdQNAHzglTrK0sdeQ7,Dominik Eulberg,317207,31,0.802,0.407,2,-15.470,1,0.0392,0.166000,0.877000,0.1110,0.1140,123.005
5,Tidalwave,Tidalwave,7wB5Ktf3MbsbNRgEDI5B80,Moksi,168196,51,0.755,0.819,0,-5.885,1,0.0330,0.013800,0.002900,0.0783,0.4870,121.972
6,Crispy Bacon,30,3PQGdUJ9GTKv0EttfvPlii,Laurent Garnier,352093,43,0.712,0.562,7,-16.716,1,0.0752,0.004910,0.796000,0.1030,0.3550,130.101
7,Epsilon Centauri,Theatre of Freaks,1vwjBiZmzNxOkskPrTvm4P,Ron Flatter,442497,38,0.695,0.613,5,-9.468,0,0.0463,0.003720,0.882000,0.1150,0.3970,122.000
8,Impact - Soulwax Remix – Edit,Impact (Soulwax Remix),1bHrUhSO0HYL1bM8QeYJww,SG Lewis,218234,46,0.759,0.837,9,-5.892,1,0.0565,0.003130,0.007400,0.1060,0.5640,116.024
9,Safe,Safe,7v2iUsHqHR1VNLODNUkTOz,Monkey Safari,501649,55,0.648,0.725,1,-8.806,1,0.0325,0.000169,0.834000,0.1070,0.6050,123.999


In [26]:
test_final_recomms

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,For an Endless Night - Jel Ford Remix,For an Endless Night (Jel Ford Remix),39KN3wgqTrGFxvWRyMW5zL,Alan Fitzpatrick,449768,38,0.751,0.649,8,-6.885,0,0.0579,0.01510,0.761,0.0976,0.1790,125.996
1,Madness To Mayhem,Oddyssey,5KNxP5oVDzSjtrpnjbFxIZ,Amtrac,416068,56,0.663,0.701,5,-8.260,1,0.0387,0.14400,0.803,0.2690,0.0373,120.000
2,Pursuit,Aleph,2jJIxT6WAIVWzDo3Ou53Q0,Gesaffelstein,247603,56,0.682,0.934,7,-4.822,1,0.0741,0.06190,0.926,0.1060,0.0708,108.982
3,u.,u.,0J7oHYxjF6Oln61wy0kP2i,niteboi,174240,53,0.463,0.653,4,-8.994,0,0.0998,0.21800,0.867,0.1110,0.0648,123.876
4,Carong,Stay,71MaaDX6bpiyUPNAkhaKvo,959er,515887,36,0.774,0.388,9,-13.995,0,0.0614,0.02120,0.799,0.1980,0.0781,107.002
5,Swallow,Amniotic,5OKoDYrRmSbPJAGxdN8UO3,Monolink,266280,44,0.677,0.756,6,-8.003,0,0.0322,0.00539,0.454,0.0942,0.3330,121.020
6,Epsilon Centauri,Theatre of Freaks,1vwjBiZmzNxOkskPrTvm4P,Ron Flatter,442497,38,0.695,0.613,5,-9.468,0,0.0463,0.00372,0.882,0.1150,0.3970,122.000
7,Run Away - Orginal Version,Romantic Thrills,3wTQ7lKiTzu1ySqmjfJ9Tu,Niconé,403284,36,0.837,0.567,11,-9.699,0,0.1040,0.02350,0.518,0.0792,0.3880,125.001
8,German Summer,Herberts Best 4,7nIpmWLpw7pnBmV1ZaMGhl,Andhim,433796,33,0.787,0.867,10,-7.957,0,0.0547,0.02400,0.814,0.1930,0.5840,122.007
9,No Place,SOLACE,6rLQZW36LaEUGWl3sWTg7v,RÜFÜS DU SOL,237986,61,0.612,0.876,4,-6.247,0,0.0315,0.01670,0.175,0.1890,0.1870,120.022


In [32]:
len(final_recomms)

49

In [33]:
final_recomms[final_recomms["track_name"].isin(redlight_df["track_name"])]

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
2,Personal Space,Personal Space / Mulholland 99,3TYNQdnAM85I6P9UegzYKo,Yotto,301295,46,0.645,0.848,7,-6.913,1,0.0351,0.00254,0.956,0.0868,0.0383,122.011
20,If I Could I Would,Euch die Uhren uns die Zeit,0wx7ZtHC12OTHIWW9hJafV,Dirty Doering,439664,55,0.893,0.543,4,-13.060,0,0.0864,0.17300,0.907,0.0806,0.1040,117.992
30,Doctor - Original,Summer Sol II,4JmLPkPHLjBN6Zt60lC64y,Sabo,481996,36,0.787,0.352,8,-14.457,0,0.1180,0.44800,0.873,0.2690,0.2510,105.010
32,Arecibo - Zebra Centauri Remix,Arecibo,263uBMSWszCKFELraxopyl,Trippin Jaguar,426017,29,0.795,0.376,2,-14.097,0,0.0448,0.13200,0.870,0.0790,0.3160,107.985


In [27]:
#filter again so tracks are not already in playlist_df
final_recomms = final_recomms[~final_recomms["track_name"].isin(redlight_df["track_name"])]
final_recomms.reset_index(drop = True, inplace = True)

In [28]:
#filter again so tracks are not already in playlist_df
test_final_recomms = test_final_recomms[~test_final_recomms["track_name"].isin(redlight_df["track_name"])]
test_final_recomms.reset_index(drop = True, inplace = True)

In [29]:
final_recomms

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,For an Endless Night - Jel Ford Remix,For an Endless Night (Jel Ford Remix),39KN3wgqTrGFxvWRyMW5zL,Alan Fitzpatrick,449768,38,0.751,0.649,8,-6.885,0,0.0579,0.015100,0.761000,0.0976,0.1790,125.996
1,AC1609 - Edit,AC1609 (Edit),0fKfg36ICmmfYBGAtqJ0Yh,Kölsch,272487,29,0.611,0.820,2,-9.247,1,0.0420,0.104000,0.927000,0.0949,0.0630,124.989
2,Up All Night - Radio Edit,Up All Night,60OOnXwxebQ4nysceVgQZX,Agoria,182787,30,0.562,0.719,11,-8.338,0,0.0328,0.060300,0.745000,0.0944,0.2260,121.009
3,Vierfleck - Recondite Remix,Mannigfaltig Remixes (Pt. 2),3i7PKdQNAHzglTrK0sdeQ7,Dominik Eulberg,317207,31,0.802,0.407,2,-15.470,1,0.0392,0.166000,0.877000,0.1110,0.1140,123.005
4,Tidalwave,Tidalwave,7wB5Ktf3MbsbNRgEDI5B80,Moksi,168196,51,0.755,0.819,0,-5.885,1,0.0330,0.013800,0.002900,0.0783,0.4870,121.972
5,Crispy Bacon,30,3PQGdUJ9GTKv0EttfvPlii,Laurent Garnier,352093,43,0.712,0.562,7,-16.716,1,0.0752,0.004910,0.796000,0.1030,0.3550,130.101
6,Epsilon Centauri,Theatre of Freaks,1vwjBiZmzNxOkskPrTvm4P,Ron Flatter,442497,38,0.695,0.613,5,-9.468,0,0.0463,0.003720,0.882000,0.1150,0.3970,122.000
7,Impact - Soulwax Remix – Edit,Impact (Soulwax Remix),1bHrUhSO0HYL1bM8QeYJww,SG Lewis,218234,46,0.759,0.837,9,-5.892,1,0.0565,0.003130,0.007400,0.1060,0.5640,116.024
8,Safe,Safe,7v2iUsHqHR1VNLODNUkTOz,Monkey Safari,501649,55,0.648,0.725,1,-8.806,1,0.0325,0.000169,0.834000,0.1070,0.6050,123.999
9,Do Not Do Me (Like Dis),Do Not Do Me (Like Dis),4DGrOp8YhMx5WocfOd0pQE,Moonbootica,194081,27,0.774,0.818,8,-6.445,1,0.0502,0.018200,0.012600,0.0884,0.4340,119.009


In [30]:
test_final_recomms

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,For an Endless Night - Jel Ford Remix,For an Endless Night (Jel Ford Remix),39KN3wgqTrGFxvWRyMW5zL,Alan Fitzpatrick,449768,38,0.751,0.649,8,-6.885,0,0.0579,0.01510,0.761,0.0976,0.1790,125.996
1,Madness To Mayhem,Oddyssey,5KNxP5oVDzSjtrpnjbFxIZ,Amtrac,416068,56,0.663,0.701,5,-8.260,1,0.0387,0.14400,0.803,0.2690,0.0373,120.000
2,Pursuit,Aleph,2jJIxT6WAIVWzDo3Ou53Q0,Gesaffelstein,247603,56,0.682,0.934,7,-4.822,1,0.0741,0.06190,0.926,0.1060,0.0708,108.982
3,u.,u.,0J7oHYxjF6Oln61wy0kP2i,niteboi,174240,53,0.463,0.653,4,-8.994,0,0.0998,0.21800,0.867,0.1110,0.0648,123.876
4,Carong,Stay,71MaaDX6bpiyUPNAkhaKvo,959er,515887,36,0.774,0.388,9,-13.995,0,0.0614,0.02120,0.799,0.1980,0.0781,107.002
5,Swallow,Amniotic,5OKoDYrRmSbPJAGxdN8UO3,Monolink,266280,44,0.677,0.756,6,-8.003,0,0.0322,0.00539,0.454,0.0942,0.3330,121.020
6,Epsilon Centauri,Theatre of Freaks,1vwjBiZmzNxOkskPrTvm4P,Ron Flatter,442497,38,0.695,0.613,5,-9.468,0,0.0463,0.00372,0.882,0.1150,0.3970,122.000
7,Run Away - Orginal Version,Romantic Thrills,3wTQ7lKiTzu1ySqmjfJ9Tu,Niconé,403284,36,0.837,0.567,11,-9.699,0,0.1040,0.02350,0.518,0.0792,0.3880,125.001
8,German Summer,Herberts Best 4,7nIpmWLpw7pnBmV1ZaMGhl,Andhim,433796,33,0.787,0.867,10,-7.957,0,0.0547,0.02400,0.814,0.1930,0.5840,122.007
9,No Place,SOLACE,6rLQZW36LaEUGWl3sWTg7v,RÜFÜS DU SOL,237986,61,0.612,0.876,4,-6.247,0,0.0315,0.01670,0.175,0.1890,0.1870,120.022


In [31]:
# sp_m.user_playlist_add_tracks(usernames[0],
#                               playlist_id="spotify:playlist:4a5UzxEPehS5Zoqh7lFREs",
#                               tracks = final_recomms["track_id"].tolist())

{'snapshot_id': 'NSw4OTM2NTc0ODA2OWY0OTc0NTQwYzBjYmNlNGRjODA0Nzg4YzNjNjAw'}

In [31]:
#add both and compare wait between to
sp_m.user_playlist_add_tracks(usernames[0],
                             playlist_id="spotify:playlist:36MtjIS6lPXT7Q97HieR9g",
                             tracks = final_recomms["track_id"].tolist())

{'snapshot_id': 'NCw1NGJlZmJhNTdiZGRmNTc4NThkYjkxODgyMjE4OTU0MGZmNDUxYWE2'}

In [32]:
sp_m.user_playlist_add_tracks(usernames[0],
                             playlist_id="spotify:playlist:1ypSXCaY044CeRD98pteRc",
                             tracks = test_final_recomms["track_id"].tolist())

{'snapshot_id': 'Miw2MzAwZTcwZjk0YTU2OTNhNGUxMmI1YjQ2ZWVhMmFmOTZmZTdhNzM3'}

In [32]:
#this playlist was good but not close enough to the original

In [26]:
# mean_song["track_name"] = "avg_song"
# mean_song["album"] = "whoknows"
# mean_song["track_id"] = "notgiven"
# mean_song["artist"] = "math"
# mean_song.reset_index(drop = True, inplace = True)

In [24]:
create_similarity_score(mean_song,final_recomms) #something doesn't work here

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [41]:
def test_filter_with_meansong(mean_song,recommendations_df, n_recommendations = 10):
    features = list(mean_song.columns[6:])
    features.remove("key")
    features.remove("mode")
    mean_song_feat = mean_song[features].values
    mean_song_scaled = MinMaxScaler().fit_transform(mean_song_feat.reshape(-1,1))
    recommendations_df_scaled = MinMaxScaler().fit_transform(recommendations_df[features])
    mean_song_scaled = mean_song_scaled.reshape(1,-1)
    sim_mean_finrecomms = cosine_similarity(mean_song_scaled,recommendations_df_scaled)[0][:]
    #sim_mean_finrecomms = sim_mean_finrecomms[0][:]
    indices = (-sim_mean_finrecomms).argsort()[:n_recommendations]
    final_recommendations = recommendations_df.iloc[indices]
    return final_recommendations

In [35]:
features = list(mean_song.columns[6:])
mean_song_feat = mean_song[features].values

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler


mean_song_scaled = StandardScaler().fit_transform(mean_song_feat.reshape(-1,1))
mean_song_scaled = mean_song_scaled.reshape(1,-1)

In [37]:
final_recomms_scaled = StandardScaler().fit_transform(final_recomms[features])
final_recomms_scaled

array([[ 0.22001416,  0.34818332,  0.70624749,  1.03922898, -0.8       ,
        -0.18940055, -0.70703119,  0.09806205, -0.31454183, -0.67529404,
         0.36807014],
       [-0.89363778,  1.27601047, -0.85894966,  0.35538708,  1.25      ,
        -0.4344784 , -0.3743085 ,  0.65941862, -0.33924372, -1.26629904,
         0.31214283],
       [-1.28341596,  0.7279956 ,  1.48884607,  0.61855909, -0.8       ,
        -0.57628445, -0.53786285,  0.04395539, -0.34381815, -0.43583512,
         0.09109946],
       [ 0.62570166, -0.96488201, -0.85894966, -1.44628446,  1.25      ,
        -0.47763676, -0.14226344,  0.49033532, -0.19194728, -1.00646064,
         0.20195437],
       [ 0.25183279,  1.27058458, -1.38068204,  1.32874715,  1.25      ,
        -0.57320171, -0.71189665, -2.46557902, -0.49111459,  0.89392613,
         0.14458307],
       [-0.09021745, -0.12386909,  0.4453813 , -1.80702409,  1.25      ,
         0.07725648, -0.74516892,  0.21642036, -0.26513806,  0.2214032 ,
         0.596

In [38]:
sim_mean_finrecomms = cosine_similarity(mean_song_scaled,final_recomms_scaled)
sim_mean_finrecomms = sim_mean_finrecomms[0][:]
indices = (-sim_mean_finrecomms).argsort()[:10]
indices

array([16, 40,  5, 26, 35, 11,  3,  0, 12, 18], dtype=int64)

In [39]:
final_recomms3 = final_recomms.iloc[indices]

In [40]:
final_recomms3

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
16,Another Place,Another Place,1cyRXghdofqkqavGpQzHZt,AK,185294,35,0.554,0.709,7,-16.472,0,0.0444,0.15000,0.830000,0.1110,0.1470,136.047
40,All We Want Is to Smile (feat. Alvaro Suarez),Buddha Bar XX,6r4Nwg6bENlhXYBCRC0WuH,Discoshaman,461693,24,0.764,0.467,8,-10.899,1,0.4200,0.45700,0.876000,0.0843,0.2900,189.991
5,Crispy Bacon,30,3PQGdUJ9GTKv0EttfvPlii,Laurent Garnier,352093,43,0.712,0.562,7,-16.716,1,0.0752,0.00491,0.796000,0.1030,0.3550,130.101
26,Feels So Good,Fear Not,2NwIZSFFruphm0aZxcwPNK,Logistics,299125,37,0.394,0.951,1,-4.396,0,0.0382,0.00122,0.003520,0.6490,0.0635,170.074
35,Southerly,Southerly,2GwpIphHopWzZgHI7m4rnZ,Tom Day,284709,32,0.503,0.778,0,-10.125,1,0.0443,0.25000,0.855000,0.1290,0.3000,138.001
11,Millenium,Eternity,6T5hiZikP7CNI2uXdWYlCg,Joachim Pastor,421935,55,0.443,0.517,6,-10.019,0,0.0542,0.03720,0.895000,0.0702,0.0739,123.978
3,Vierfleck - Recondite Remix,Mannigfaltig Remixes (Pt. 2),3i7PKdQNAHzglTrK0sdeQ7,Dominik Eulberg,317207,31,0.802,0.407,2,-15.470,1,0.0392,0.16600,0.877000,0.1110,0.1140,123.005
0,For an Endless Night - Jel Ford Remix,For an Endless Night (Jel Ford Remix),39KN3wgqTrGFxvWRyMW5zL,Alan Fitzpatrick,449768,38,0.751,0.649,8,-6.885,0,0.0579,0.01510,0.761000,0.0976,0.1790,125.996
12,Ete Pluvieux - Original Mix,Get Lost V mixed by Acid Pauli,1OV89RQpZ1Ok3HWBwNVWBX,Dirty Doering,537499,35,0.845,0.310,11,-12.621,0,0.0515,0.11800,0.929000,0.0894,0.3820,123.990
18,Apart,After Forever,1SIU0svdzBXZInOzmeY3e7,Kasper Bjørke,345450,34,0.629,0.656,0,-8.396,1,0.0290,0.01030,0.000656,0.0953,0.1120,123.956


- Some songs are "correct" and are also part of the manual selection, 2 however are wrong, and contain to much text.
- manual filtering  (before mean song) based on attributes such as instrumentalness may be helpful (as an option)

In [33]:
final_recomms_test = filter_with_meansong(mean_song,final_recomms)

AttributeError: 'numpy.ndarray' object has no attribute 'remove'

In [51]:
assert final_recomms3.equals(final_recomms_test) #function works

In [43]:
testScaling_final_recomms = test_filter_with_meansong(mean_song,test_final_recomms)

In [44]:
testScaling_final_recomms

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
36,All We Want Is to Smile (feat. Alvaro Suarez),Buddha Bar XX,6r4Nwg6bENlhXYBCRC0WuH,Discoshaman,461693,24,0.764,0.467,8,-10.899,1,0.4200,0.4570,0.876,0.0843,0.290,189.991
29,Southerly,Southerly,2GwpIphHopWzZgHI7m4rnZ,Tom Day,284709,32,0.503,0.778,0,-10.125,1,0.0443,0.2500,0.855,0.1290,0.300,138.001
7,Run Away - Orginal Version,Romantic Thrills,3wTQ7lKiTzu1ySqmjfJ9Tu,Niconé,403284,36,0.837,0.567,11,-9.699,0,0.1040,0.0235,0.518,0.0792,0.388,125.001
32,Satisfied - Edit,Silver Linings,378c4bnvmnfAoMTSIOveDX,Catching Flies,223336,50,0.744,0.593,4,-8.765,0,0.1070,0.0482,0.300,0.2640,0.120,122.050
16,Maybe Not - Rodriguez Jr. Extended Remix,Maybe Not (Rodriguez Jr. Remix),4b9xA8i8hRFNCe745q6UEY,Jan Blomqvist,389488,21,0.629,0.606,1,-12.952,1,0.0393,0.0427,0.886,0.1150,0.158,121.995
34,Under The Moon - Cassius Remix - Edit,Under The Moon (Remixes),2dAcb7rIb2EeFsz8hebdue,Claptone,233703,48,0.780,0.510,5,-10.811,0,0.0436,0.0095,0.695,0.1480,0.122,120.928
31,Zomer,Zomer,75AiEL5QmWKer1hqrAJPpf,Bakermat,408133,45,0.815,0.468,8,-9.370,0,0.0854,0.0114,0.816,0.0742,0.484,126.003
24,Netzwerk - Super Flu ReDings,Netzwerk (Super Flu ReDings),491kxjPTC7c6qQOxqSiDO5,Klangkarussell,594150,31,0.670,0.603,8,-12.553,0,0.0383,0.0525,0.895,0.2030,0.070,121.004
0,For an Endless Night - Jel Ford Remix,For an Endless Night (Jel Ford Remix),39KN3wgqTrGFxvWRyMW5zL,Alan Fitzpatrick,449768,38,0.751,0.649,8,-6.885,0,0.0579,0.0151,0.761,0.0976,0.179,125.996
27,Liebe Ist...,Herzblut Pur,3pFPOLhjP0SzePQEPxGPxw,Stephan Bodzin,432756,43,0.941,0.413,7,-9.372,1,0.1290,0.1540,0.856,0.1080,0.543,126.987


In [55]:
test_final_recomms[test_final_recomms["track_name"] == "Pandora"]

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
30,Pandora,Pandora,2kscYCgdy8FP8D92hgsVIy,Renga Weh,355084,36,0.805,0.457,1,-12.792,1,0.1,0.00551,0.908,0.106,0.388,120.011


In [52]:
sp_m.user_playlist_add_tracks(usernames[0],
                              playlist_id="spotify:playlist:4XP9wRGPImiYYiGtsB6Dd3",
                              tracks = final_recomms3["track_id"].tolist())

{'snapshot_id': 'MjEsMzQ4ZmFjNjhkZDQ0YWQ1YjRkODI3YjQ3Zjg4NmY2ODc4MDg1NWY4Yg=='}

In [ ]:
#this method seemed to work quite well